In [1]:
import pandas as pd

In [2]:
dataset=pd.read_csv('retail_bakery_transactions.csv')

In [3]:
dataset.columns=dataset.columns.str.lower()

In [4]:
dataset=dataset.applymap(lambda s:s.lower() if type(s)==str else s)

In [5]:
dataset.head()

,transaction,item,date_time,period_day,weekday_weekend
0,1,bread,30-10-2016 09:58,morning,weekend
1,2,scandinavian,30-10-2016 10:05,morning,weekend
2,2,scandinavian,30-10-2016 10:05,morning,weekend
3,3,hot chocolate,30-10-2016 10:07,morning,weekend
4,3,jam,30-10-2016 10:07,morning,weekend


In [6]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20507 entries, 0 to 20506
Data columns (total 5 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   transaction      20507 non-null  int64 
 1   item             20507 non-null  object
 2   date_time        20507 non-null  object
 3   period_day       20507 non-null  object
 4   weekday_weekend  20507 non-null  object
dtypes: int64(1), object(4)
memory usage: 801.2+ KB


In [7]:
dataset.isnull().sum()

transaction        0
item               0
date_time          0
period_day         0
weekday_weekend    0
dtype: int64

In [8]:
transaction_list=dataset.groupby(['transaction','date_time'])['item'].apply(lambda x:list(x))

In [9]:
transaction_list.head()

transaction  date_time       
1            30-10-2016 09:58                          [bread]
2            30-10-2016 10:05     [scandinavian, scandinavian]
3            30-10-2016 10:07    [hot chocolate, jam, cookies]
4            30-10-2016 10:08                         [muffin]
5            30-10-2016 10:13          [coffee, pastry, bread]
Name: item, dtype: object

In [10]:
df=transaction_list.values.tolist()

In [11]:
df[:5]

[['bread'],
 ['scandinavian', 'scandinavian'],
 ['hot chocolate', 'jam', 'cookies'],
 ['muffin'],
 ['coffee', 'pastry', 'bread']]

In [12]:
from mlxtend.preprocessing import TransactionEncoder


In [13]:
encoder=TransactionEncoder().fit(df)

In [14]:
onehot=encoder.transform(df)

In [15]:
transf_df=pd.DataFrame(onehot,columns=encoder.columns_)

In [16]:
transf_df.head()

,adjustment,afternoon with the baker,alfajores,argentina night,art tray,bacon,baguette,bakewell,bare popcorn,basket,...,the bart,the nomad,tiffin,toast,truffles,tshirt,valentine's card,vegan feast,vegan mincepie,victorian sponge
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [17]:
from mlxtend.frequent_patterns import fpgrowth

In [25]:
frequent_itemsets=fpgrowth(transf_df,min_support=0.05,use_colnames=True)

In [26]:
frequent_itemsets.sort_values('support',ascending=False)

,support,itemsets
3,0.478394,(coffee)
0,0.327205,(bread)
6,0.142631,(tea)
7,0.103856,(cake)
9,0.090016,"(coffee, bread)"
4,0.086107,(pastry)
8,0.071844,(sandwich)
5,0.061807,(medialuna)
1,0.058320,(hot chocolate)
10,0.054728,"(coffee, cake)"


In [27]:
frequent_itemsets['length']=frequent_itemsets['itemsets'].apply(lambda x:len(x))
frequent_itemsets

,support,itemsets,length
0,0.327205,(bread),1
1,0.058320,(hot chocolate),1
2,0.054411,(cookies),1
3,0.478394,(coffee),1
4,0.086107,(pastry),1
5,0.061807,(medialuna),1
6,0.142631,(tea),1
7,0.103856,(cake),1
8,0.071844,(sandwich),1
9,0.090016,"(coffee, bread)",2


In [28]:
frequent_itemsets['length'].value_counts()

1    9
2    2
Name: length, dtype: int64

In [29]:
from mlxtend.frequent_patterns import association_rules

In [30]:
rules=association_rules(frequent_itemsets,metric='lift',min_threshold=1.0)

In [31]:
rules.head()

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(coffee),(cake),0.478394,0.103856,0.054728,0.114399,1.101515,0.005044,1.011905
1,(cake),(coffee),0.103856,0.478394,0.054728,0.526958,1.101515,0.005044,1.102664
